# Calculating the proportion of population, cases, and deaths in largest US metro areas

This notebook aggregates COVID-19 cases and death counts to the level of Census-defined [Core-Based Statistical Areas](https://www.census.gov/topics/housing/housing-patterns/about/core-based-statistical-areas.html) (i.e., metropolitan and micropolitan ares), and then calculates the proportion of population, cases, and deaths within the 15 largest, in relation to US totals.

## Load CBSA and COVID data

In [1]:
import pandas as pd
import numpy as np

CBSA information — their names, populations, and constituent counties — comes from the Census Bureau's ["delineation files"](https://www.census.gov/geographies/reference-files/time-series/demo/metro-micro/delineation-files.html) and [Population Estimates API](https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html):

In [2]:
cbsas = (
    pd.read_csv(
        "../data/county-data/cbsas.csv",
        dtype = {
            "cbsa_code": str,
            "population": int,
        }
    )
)

cbsas.head()

,cbsa_code,cbsa_title,metro_micro,population
0,10100,"Aberdeen, SD",Micropolitan Statistical Area,42668
1,10140,"Aberdeen, WA",Micropolitan Statistical Area,75061
2,10180,"Abilene, TX",Metropolitan Statistical Area,172060
3,10220,"Ada, OK",Micropolitan Statistical Area,38284
4,10300,"Adrian, MI",Micropolitan Statistical Area,98451


In [3]:
cbsa_counties = (
    pd.read_csv(
        "../data/county-data/cbsa-counties.csv",
        dtype = {
            "cbsa_code": str,
            "county_fips": str,
        }
    )
)

cbsa_counties.head()

,cbsa_code,county_fips,county_name,state_name
0,10100,46013,Brown County,South Dakota
1,10100,46045,Edmunds County,South Dakota
2,10140,53027,Grays Harbor County,Washington
3,10180,48059,Callahan County,Texas
4,10180,48253,Jones County,Texas


County-level COVID-19 counts [come from the New York Times](https://github.com/nytimes/covid-19-data):

In [4]:
nyt_counties = (
    pd.read_csv(
        "../data/county-data/nyt-county-counts.csv",
        dtype = {
            "fips": str
        }
    )
    .rename(columns = {
        "fips": "county_fips"
    })
)

nyt_counties.head()

,date,county,state,county_fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


## Select only the most recent counts for each county

In [5]:
nyt_counties_latest = (
    nyt_counties
    .sort_values([ "date" ], ascending = False)
    .drop_duplicates(subset = [ "state", "county" ])
    .loc[lambda df: df["date"] == df["date"].max()]
    .sort_values("deaths", ascending = False)
)

nyt_counties_latest.head()

,date,county,state,county_fips,cases,deaths
114279,2020-05-04,New York City,New York,NaN,175661,13207
114278,2020-05-04,Nassau,New York,36059,36965,2221
113810,2020-05-04,Wayne,Michigan,26163,17314,1924
113147,2020-05-04,Cook,Illinois,17031,43715,1802
114298,2020-05-04,Suffolk,New York,36103,35077,1325


In [6]:
(
    nyt_counties_latest
    [[
        "cases",
        "deaths"
    ]]
    .sum()
)

cases     1180527
deaths      63213
dtype: int64

## Identify non-county entities in NYT data and move them to intra-CBSA counties

See the ["Geographic Exceptions" section](https://github.com/nytimes/covid-19-data#geographic-exceptions) of the New York Times' methodology for background.

In [7]:
(
    nyt_counties_latest
    .loc[lambda df: df["county_fips"].isnull()]
    .loc[lambda df: df["county"] != "Unknown"]
)

,date,county,state,county_fips,cases,deaths
114279,2020-05-04,New York City,New York,NaN,175661,13207
114020,2020-05-04,Kansas City,Missouri,NaN,659,16


In [8]:
county_counts = (
    nyt_counties_latest
    .assign(
        county_fips = lambda df: (
            df["county"]
            .apply({
                "New York City": "36061", # New York City -> Manhattan
                "Kansas City": "29095", # Kansas City -> Jackson County
            }.get)
            .fillna(df["county_fips"])
        )
    )
    .groupby([ "county_fips" ])
    [[
        "cases",
        "deaths"
    ]]
    .sum()
    .reset_index()
)

county_counts.head()

,county_fips,cases,deaths
0,01001,53,3
1,01003,188,4
2,01005,45,1
3,01007,42,0
4,01009,40,0


## Aggregated data to the CBSA level, by merging the NYT and Census datasets

In [9]:
cbsa_counts = (
    county_counts
    .merge(
        cbsa_counties
        [[
            "cbsa_code",
            "county_fips",
        ]],
        how = "left",
        validate = "1:1"
    )
    .groupby("cbsa_code")
    [[
        "cbsa_code",
        "cases",
        "deaths",
    ]]
    .sum()
    .reset_index()
    .merge(
        cbsas,
        how = "left",
        on = "cbsa_code",
        validate = "1:1",
    )
)

cbsa_counts.head()

,cbsa_code,cases,deaths,cbsa_title,metro_micro,population
0,10100,65,0,"Aberdeen, SD",Micropolitan Statistical Area,42668
1,10140,12,0,"Aberdeen, WA",Micropolitan Statistical Area,75061
2,10180,429,5,"Abilene, TX",Metropolitan Statistical Area,172060
3,10220,10,2,"Ada, OK",Micropolitan Statistical Area,38284
4,10300,110,2,"Adrian, MI",Micropolitan Statistical Area,98451


## Identify the 15 largest CBSAs / metro areas

In [10]:
largest_metros = (
    cbsa_counts
    .nlargest(15, "population")
)

largest_metros

,cbsa_code,cases,deaths,cbsa_title,metro_micro,population
598,35620,402032,25513,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan Statistical Area,19216182
491,31080,29076,1313,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan Statistical Area,13214799
160,16980,61243,2550,"Chicago-Naperville-Elgin, IL-IN-WI",Metropolitan Statistical Area,9458539
207,19100,9164,254,"Dallas-Fort Worth-Arlington, TX",Metropolitan Statistical Area,7573136
382,26420,10144,219,"Houston-The Woodlands-Sugar Land, TX",Metropolitan Statistical Area,7066141
880,47900,31284,1259,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,6280487
540,33100,21783,782,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan Statistical Area,6166488
654,37980,41021,2211,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,6102434
45,12060,14499,560,"Atlanta-Sandy Springs-Alpharetta, GA",Metropolitan Statistical Area,6020364
655,38060,5219,181,"Phoenix-Mesa-Chandler, AZ",Metropolitan Statistical Area,4948203


## Calculate those CBSAs' total populations, cases, and deaths — and their proportion relative to entire US

### Proportion of US population

US total figure [via Census Population Estimates](https://www.census.gov/newsroom/press-releases/2019/popest-nation.html)

In [11]:
largest_metros["population"].sum()

108601700

In [12]:
(largest_metros["population"].sum() / 328239523).round(3)

0.331

### Proportion of cases and deaths

In [13]:
largest_metros[["cases", "deaths"]].sum()

cases     732534
deaths     42365
dtype: int64

In [14]:
(largest_metros[[ "cases", "deaths" ]].sum() / nyt_counties_latest[[ "cases", "deaths" ]].sum()).round(3)

cases     0.621
deaths    0.670
dtype: float64

---